# JL - DMBA - PPP Project

## Libraries


In [3]:
import numpy as np
import pandas as pd

import random
import os 
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings("ignore")

# openpyxl: nstall this to open Excel files 
# !pip install openpyxl
import openpyxl

## Files

In [4]:
# Use pd.read_excel to read the Excel file
ppp_data_dict_df = pd.read_csv('archive/ppp_data_dict.csv')
ppp_over_150k_df = pd.read_csv('archive/ppp_over_150k.csv')
ppp_fraud_df = pd.read_csv('archive/ppp_fraud_cases.csv')

## Basic EDA
- Note: the basic EDA is can refer to kaggle ipynb file - if want can get raw data from the origin source instead
    - basic na checks, fill na and new properties created (listed in table below))


datadict:
('https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/aab8e9f9-36d1-42e1-b3ba-e59c79f1d7f0/download/ppp-data-dictionary.xlsx')

dataset:
('https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/501af711-1c91-477a-80ce-bf6428eb9253/download/public_150k_plus_220403.csv')
fraud case dataset is written by kaggle owner

| ID | Variable Name                        | Description                                                               |
|----|--------------------------------------|---------------------------------------------------------------------------|
| 53 | ApprovalDiff                         | Difference between CurrentApprovalAmount and InitialApprovalAmount        |
| 54 | NotForgivenAmount                    | Loan amount that has not been forgiven                                      |
| 55 | ForgivenPercentage                   | Percentage of the CurrentApprovalAmount that has been forgiven             |
| 56 | TOTAL_PROCEED                        | Sum of all prior PROCEED columns                                             |
| 57 | PROCEED_Diff                         | Difference between CurrentApprovalAmount and TOTAL_PROCEED                   |
| 58 | UTILITIES_PROCEED_pct                | UTILITIES_PROCEED as a percentage of CurrentApprovalAmount                   |
| 59 | PAYROLL_PROCEED_pct                  | PAYROLL_PROCEED as a percentage of CurrentApprovalAmount                     |
| 60 | MORTGAGE_INTEREST_PROCEED_pct        | MORTGAGE_INTEREST_PROCEED as a percentage of CurrentApprovalAmount           |
| 61 | RENT_PROCEED_pct                     | RENT_PROCEED as a percentage of CurrentApprovalAmount                        |
| 62 | REFINANCE_EIDL_PROCEED_pct           | REFINANCE_EIDL_PROCEED as a percentage of CurrentApprovalAmount              |
| 63 | HEALTH_CARE_PROCEED_pct              | HEALTH_CARE_PROCEED as a percentage of CurrentApprovalAmount                 |
| 64 | DEBT_INTEREST_PROCEED_pct            | DEBT_INTEREST_PROCEED as a percentage of CurrentApprovalAmount               |
| 65 | PROCEED_Per_Job                      | PAYROLL_PROCEED divided by JobsReported                                       |


In [5]:
ppp_data_dict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Field Name         66 non-null     object
 1   Field Description  66 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [6]:
# Set options to display all rows and full content of cells
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
display(ppp_data_dict_df)

,Field Name,Field Description
0,LoanNumber,Loan Number (unique identifier)
1,DateApproved,Loan Funded Date
2,SBAOfficeCode,SBA Origination Office Code
3,ProcessingMethod,Loan Delivery Method (PPP for first draw; PPS for second draw)
4,BorrowerName,Borrower Name
5,BorrowerAddress,Borrower Street Address
6,BorrowerCity,Borrower City
7,BorrowerState,Borrower State
8,BorrowerZip,Borrower Zip Code
9,LoanStatusDate,Loan Status Date\n- Loan Status Date is blank when the loan is disbursed but not Paid In Full or Charged Off


In [33]:
ppp_over_150k_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968532 entries, 0 to 968531
Data columns (total 66 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   LoanNumber                     968532 non-null  int64  
 1   DateApproved                   968532 non-null  object 
 2   SBAOfficeCode                  968532 non-null  int64  
 3   ProcessingMethod               968532 non-null  object 
 4   BorrowerName                   968525 non-null  object 
 5   BorrowerAddress                968532 non-null  object 
 6   BorrowerCity                   968532 non-null  object 
 7   BorrowerState                  968532 non-null  object 
 8   BorrowerZip                    968532 non-null  object 
 9   LoanStatusDate                 968532 non-null  object 
 10  LoanStatus                     968532 non-null  object 
 11  Term                           968532 non-null  int64  
 12  SBAGuarantyPercentage         

In [7]:
ppp_over_150k_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,TOTAL_PROCEED,PROCEED_Diff,UTILITIES_PROCEED_pct,PAYROLL_PROCEED_pct,MORTGAGE_INTEREST_PROCEED_pct,RENT_PROCEED_pct,REFINANCE_EIDL_PROCEED_pct,HEALTH_CARE_PROCEED_pct,DEBT_INTEREST_PROCEED_pct,PROCEED_Per_Job
0,9547507704,2020-05-01,464,PPP,sumter coatings inc,2410 highway 15 south,sumter,UNK,29150-9662,2020-12-18,...,769358.78,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12409.01
1,9777677704,2020-05-01,464,PPP,pleasant places inc,7684 southrail road,north charleston,UNK,29420-9000,2021-09-28,...,736927.79,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,10094.90
2,5791407702,2020-05-01,1013,PPP,boyer childrens clinic,1850 boyer ave e,seattle,UNK,98112-2922,2021-03-17,...,691355.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9218.07
3,6223567700,2020-05-01,920,PPP,kirtley construction inc,1661 martin ranch rd,san bernardino,UNK,92407-1740,2021-10-16,...,499871.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,23803.38
4,9662437702,2020-05-01,101,PPP,aero box llc,unknown,unknown,UNK,0,2021-08-17,...,367437.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,14697.48


In [13]:
def match_variables(df, variable_1, variable_2):
  """Matches two variables in a dataset.

  Args:
    df: A Pandas DataFrame.
    variable_1: The name of the first variable to match.
    variable_2: The name of the second variable to match.

  Returns:
    A Pandas DataFrame containing the rows where the two variables match.
  """

  dont_matched_df = df[df[variable_1] != df[variable_2]]
  
  return dont_matched_df[[variable_1, variable_2]]


match_variables(ppp_over_150k_df, 'BorrowerCity', 'ProjectCity')


,BorrowerCity,ProjectCity
3178,atmore,altmore
3302,henagar,henegar
6084,hoover,birmingham
10578,mountain brook,birmingham
15708,maumelle,north little rock
20444,tubac,tuabc
22154,chandler,mansfield
22976,flagstaff,tusayan
25595,tucson,leander
36416,oxnard,king city


In [20]:
# Get the number of rows in the DataFrame
n_rows = len(ppp_over_150k_df)

# Generate a random sample of 100 rows
random_rows = random.sample(range(n_rows), 100)

# Create a new DataFrame with the randomly selected rows
ppp_over_150k_df_sample = ppp_over_150k_df.iloc[random_rows]

# Convert the DataFrame to Excel
ppp_over_150k_df_sample.to_excel("ppp_over_150k_df_sample.xlsx")

In [39]:
unique_values = ppp_over_150k_df['ProcessingMethod'].unique()
print(unique_values)

ProcessingMethod_value_counts = ppp_over_150k_df['ProcessingMethod'].value_counts()
print(ProcessingMethod_value_counts)

LoanStatus_value_counts = ppp_over_150k_df['LoanStatus'].value_counts()
print(LoanStatus_value_counts)

SBAGuarantyPercentage_value_counts = ppp_over_150k_df['SBAGuarantyPercentage'].value_counts()
print(SBAGuarantyPercentage_value_counts)

ServicingLenderLocationID_value_counts = ppp_over_150k_df['ServicingLenderLocationID'].nunique()
print('ServicingLenderLocationID_unique_value_counts  = '+ str(ServicingLenderLocationID_value_counts))


ServicingLenderName_value_counts = ppp_over_150k_df['ServicingLenderName'].nunique()
print('ServicingLenderName_unique_value_counts = '+ str(ServicingLenderName_value_counts))

ServicingLenderAddress_value_counts = ppp_over_150k_df['ServicingLenderAddress'].nunique()
print('ServicingLenderAddress_unique_value_counts = '+ str(ServicingLenderAddress_value_counts))

ServicingLenderCity_value_counts = ppp_over_150k_df['ServicingLenderCity'].nunique()
print('ServicingLenderCity_unique_value_counts = '+ str(ServicingLenderCity_value_counts))

ServicingLenderState_value_counts = ppp_over_150k_df['ServicingLenderState'].nunique()
print('ServicingLenderState_unique_value_counts = '+ str(ServicingLenderState_value_counts))

['PPP' 'PPS']
ProcessingMethod
PPP    676873
PPS    291659
Name: count, dtype: int64
LoanStatus
Paid in Full           835009
Exemption 4            133516
Active Un-Disbursed         7
Name: count, dtype: int64
SBAGuarantyPercentage
100    968532
Name: count, dtype: int64
ServicingLenderLocationID_unique_value_counts  = 4761
ServicingLenderName_unique_value_counts = 4246
ServicingLenderAddress_unique_value_counts = 4546
ServicingLenderCity_unique_value_counts = 2725
ServicingLenderState_unique_value_counts = 55


In [27]:
ppp_over_150k_df.describe()

,LoanNumber,SBAOfficeCode,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,ServicingLenderLocationID,JobsReported,NAICSCode,...,TOTAL_PROCEED,PROCEED_Diff,UTILITIES_PROCEED_pct,PAYROLL_PROCEED_pct,MORTGAGE_INTEREST_PROCEED_pct,RENT_PROCEED_pct,REFINANCE_EIDL_PROCEED_pct,HEALTH_CARE_PROCEED_pct,DEBT_INTEREST_PROCEED_pct,PROCEED_Per_Job
count,9.685320e+05,968532.000000,968532.000000,968532.0,9.685320e+05,9.685320e+05,9.685320e+05,968532.000000,968532.000000,968532.000000,...,9.685320e+05,9.685320e+05,968532.000000,968532.000000,968532.000000,968532.000000,968532.000000,968532.000000,968532.000000,968532.0000
mean,5.427133e+09,571.520066,36.297555,100.0,5.322552e+05,5.306236e+05,1.207811e+01,107924.839664,51.886761,508303.446845,...,5.306214e+05,2.218373e+00,0.009156,0.971467,0.003424,0.010715,0.000412,0.004105,0.000716,inf
std,2.551320e+09,263.025314,17.214728,0.0,7.442512e+05,7.378869e+05,4.776398e+03,132637.319901,67.551868,185421.574560,...,7.378872e+05,5.341029e+02,0.047811,0.085583,0.029104,0.041067,0.016914,0.023227,0.010371,NaN
min,1.000007e+09,101.000000,0.000000,100.0,0.000000e+00,1.500000e+05,0.000000e+00,20.000000,0.000000,81.000000,...,9.480000e+04,-1.499700e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
25%,3.271096e+09,373.000000,24.000000,100.0,2.002000e+05,2.004000e+05,0.000000e+00,30804.000000,18.000000,335999.000000,...,2.004000e+05,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7718.4450
50%,5.400677e+09,515.000000,24.000000,100.0,2.951788e+05,2.950000e+05,0.000000e+00,57328.000000,30.000000,541211.000000,...,2.950000e+05,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11170.5900
75%,7.546310e+09,811.000000,60.000000,100.0,5.402000e+05,5.393898e+05,0.000000e+00,116998.000000,54.000000,622110.000000,...,5.393858e+05,5.820766e-11,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15074.9475
max,9.999007e+09,1094.000000,120.000000,100.0,1.000000e+07,1.000000e+07,3.397100e+06,533479.000000,500.000000,999990.000000,...,1.000000e+07,3.237070e+05,1.000000,1.041000,1.000000,1.000000,1.000000,1.000000,1.000000,inf


In [28]:
ppp_fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Company   100 non-null    object
 1   State     100 non-null    object
 2   Source    100 non-null    object
 3   Suspects  100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [ ]:
ppp_fraud_df.head()

In [ ]:
ppp_fraud_df.describe()

## Visualisations


In [ ]:
def get_numeric_variables(df):
    """
    Get all numeric variables from a DataFrame.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame.
        
    Returns:
        pd.DataFrame: DataFrame containing only numeric variables.
    """
    numeric_variables = df.select_dtypes(include=['number'])
    return numeric_variables

In [ ]:
%matplotlib inline

ppp_over_150k_df.hist(column = ['ApprovalDiff',	'NotForgivenAmount', 'ForgivenPercentage',	'TOTAL_PROCEED','PROCEED_Diff',	'UTILITIES_PROCEED_pct',	'PAYROLL_PROCEED_pct',	'MORTGAGE_INTEREST_PROCEED_pct',	'RENT_PROCEED_pct',	'REFINANCE_EIDL_PROCEED_pct',	'HEALTH_CARE_PROCEED_pct',	'DEBT_INTEREST_PROCEED_pct',	'PROCEED_Per_Job'],figsize = (20,25)) # "figsize" controls the size of the histogram
plt.tight_layout() # to display multiple histogram nicely
plt.show()



In [ ]:
# to get numeric vairables of the df only for visualisation
num_df = get_numeric_variables(ppp_over_150k_df)
num_df.drop(['LoanNumber',	'SBAOfficeCode','ServicingLenderLocationID','NAICSCode','OriginatingLenderLocationID'],axis=1,inplace=True)
num_df.head()

#### Histogram

In [ ]:
num_df.hist(figsize = (30,30)) # "figsize" controls the size of the histogram
plt.tight_layout() # to display multiple histogram nicely
plt.show()

#### Heatmap

In [ ]:
num_df.corr()
plt.figure(figsize=(40,40))
sns.heatmap(num_df.corr(), annot=True)
# Annot attribute : annotate cells with value

# Display the heatmap
plt.show()

#### Correlation Matrix

In [ ]:
# Calculate the correlation matrix
correlation_matrix = num_df.corr()

# Set a threshold for high correlation (e.g., 0.7)
threshold = 0.7

# Find pairs with correlation above the threshold
high_correlation_pairs = (correlation_matrix.abs() > threshold) & (correlation_matrix.abs() < 1)

# Get the variable pairs with high correlation
high_correlation_variables = []

# Get the variable pairs with high correlation
high_correlation_variables = []

for col in high_correlation_pairs.columns:
    for col2 in high_correlation_pairs.index:
        if high_correlation_pairs.loc[col2, col] and (col2, col) not in high_correlation_variables:
            high_correlation_variables.append((col, col2))

# Print the pairs of highly correlated variables
for pair in high_correlation_variables:
    print(pair)

#### Covariance & Correlation

In [ ]:
# Compute the covarariance and correlation
print("Covariance: ", num_df.cov())
print("Correlation: ", num_df.corr())

In [ ]:
# Get columns with non-finite values
non_finite_columns = num_df.columns[~np.isfinite(num_df).all()].tolist()

# Print the columns with non-finite values
if non_finite_columns:
    print(f"The following columns have non-finite values: {', '.join(non_finite_columns)}")
else:
    print("No columns have non-finite values.")



Notes: jobs reported since to be the most spcial correlation compared to the rest: but it wld make sense too that increase job--> increase amt etc

In [ ]:
num_df.drop('PROCEED_Per_Job',axis=1,inplace=True)
sns.clustermap(num_df.corr(), annot=True)
plt.show()

#### Boxplot

In [ ]:
# Plot boxplots for all attributes
num_df.boxplot(figsize=(40, 6))
plt.title("Boxplots of Attributes")
plt.ylabel("Values")
plt.show()

In [ ]:
attributes_to_normalize = num_df.skew()
# To identify vars which need transformation


### Data Pre-processing

In [ ]:

# Assuming num_df is your DataFrame and you've computed the skewness values
skew_threshold_min = -2  # New skew threshold
skew_threshold_max = 5.0  # New skew threshold

# Get a list of attributes to normalize based on skewness
attributes_to_normalize1 = num_df.skew().index[abs(num_df.skew()) > skew_threshold_max].tolist()
attributes_to_normalize2 = num_df.skew().index[abs(num_df.skew()) < skew_threshold_min].tolist()

# Combine the lists
attributes_to_normalize = attributes_to_normalize1 + attributes_to_normalize2
print("Attributes to Normalize:")
print(attributes_to_normalize)


#### Transformation

In [ ]:
from sklearn.preprocessing import StandardScaler

data_to_normalize = num_df[attributes_to_normalize]

# Perform z-score normalization
scaler = StandardScaler()
normalized_data = scaler.fit_transform(data_to_normalize)

# Create a new DataFrame with the normalized values
normalized_df = pd.DataFrame(normalized_data, columns=attributes_to_normalize)


# Create Dataframe with blanks filled
df1 = normalized_df.fillna(normalized_df.mean())

# Display the normalized DataFrame
print("Normalized Data (z-score normalization):")
print(df1)

In [ ]:
# Assuming 'df' is your DataFrame
non_numeric_variables = ppp_over_150k_df.select_dtypes(exclude=['number']).columns.tolist()

print("Non-Numeric Variables:")
print(non_numeric_variables)

#### Categorical Variable Handling Considerations

### Date Variables
- **DateApproved, LoanStatusDate, ForgivenessDate**
  - These variables represent dates and are not suitable for conversion to dummy variables. Consider extracting relevant date features for analysis.

### Categorical Variables
- **ProcessingMethod, LoanStatus, RuralUrbanIndicator, HubzoneIndicator, LMIIndicator, BusinessAgeDescription, CD, Race, Ethnicity, BusinessType, OriginatingLender, Gender, Veteran, NonProfit**
  - These are categorical variables. Creating dummy variables is a common approach. Consider the cardinality (number of unique categories) before creating dummies.

### Variables with Potentially High Cardinality
- **BorrowerName, BorrowerAddress, BorrowerCity, BorrowerState, BorrowerZip, FranchiseName, ServicingLenderName, ServicingLenderAddress, ServicingLenderCity, ServicingLenderState, ServicingLenderZip, ProjectCity, ProjectCountyName, ProjectState, ProjectZip, OriginatingLenderCity, OriginatingLenderState**
  - These variables may have a very large number of unique categories. Consider techniques like grouping or consolidation before creating dummy variables.

---

**Note**: currently i just dummy all except dates, some names, address but can look into the 3rd catgory variables tbh espcially the address, can catgorise according to zip or just remove the specific since state is give mostly. 

In [ ]:
#excluded:  'DateApproved', 'ForgivenessDate', 'LoanStatusDate', 'BorrowerName' 'BorrowerZip', 'BorrowerAddress', 'ServicingLenderZip', 'ProjectZip' ,'OriginatingLender',
data1 = pd.get_dummies(ppp_over_150k_df, columns = ['ProcessingMethod', 'BorrowerCity', 'BorrowerState', 'LoanStatus', 'FranchiseName', 'ServicingLenderName', 'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState', 'RuralUrbanIndicator', 'HubzoneIndicator', 'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity', 'ProjectCountyName', 'ProjectState', 'CD', 'Race', 'Ethnicity', 'BusinessType', 'OriginatingLenderCity', 'OriginatingLenderState', 'Gender', 'Veteran', 'NonProfit'], drop_first= True)
data1

# When drop_first=True, the first column of each category is dropped, resulting in one less dummy variable than the number of categories to avoid the issue of multicollinearity where the presence of one dummy variable can be predicted perfectly by the presence or absence of the others.
ppp_over_150k_df1 = pd.get_dummies(ppp_over_150k_df["survived"])
ppp_over_150k_df_new = pd.concat([ppp_over_150k_df,titanic1],axis =1)

